In [14]:
!python --version
!pip -q install pandas numpy scikit-learn plotly

Python 3.12.12


Imports and Data

In [15]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler

RANDOM_STATE = 42
REQUIRED_COLS = ["Top queries", "Clicks", "Impressions", "CTR", "Position"]

In [16]:
df = pd.read_csv("Queries.csv")
df.head()

,Top queries,Clicks,Impressions,CTR,Position
0,number guessing game python,5223,14578,35.83%,1.61
1,thecleverprogrammer,2809,3456,81.28%,1.02
2,python projects with source code,2077,73380,2.83%,5.94
3,classification report in machine learning,2012,4959,40.57%,1.28
4,the clever programmer,1931,2528,76.38%,1.09


Cleaning

In [17]:
def clean_df(df: pd.DataFrame) -> pd.DataFrame:
    # strip column names
    df = df.rename(columns={c: c.strip() for c in df.columns})
    # try to match required columns case-insensitively
    lower_map = {c.lower(): c for c in df.columns}
    fixed = {}
    alts = {
        "top queries": ["query","queries","keyword","search term","top query"],
        "clicks": ["click","total_clicks"],
        "impressions": ["impr","impressions_total","total_impressions"],
        "ctr": ["click_through_rate","click-through rate","ctr (%)","ctr%"],
        "position": ["avg position","avg_position","rank","avg rank","avg_ranking"]
    }
    for col in REQUIRED_COLS:
        key = col.lower()
        if key in lower_map:
            fixed[col] = lower_map[key]
        else:
            matched = None
            for alt in alts.get(key, []):
                if alt in lower_map:
                    matched = lower_map[alt]
                    break
            if matched is None:
                raise ValueError(f"Missing required column: {col}")
            fixed[col] = matched

    df = df[[fixed[c] for c in REQUIRED_COLS]].copy()
    df.columns = REQUIRED_COLS

    # parse CTR like "35.8%" or 0.358
    def parse_ctr(x):
        if isinstance(x, str):
            x = x.strip().replace('%', '')
            x = float(x) if x else np.nan
            return x / 100.0
        try:
            x = float(x)
            return x/100.0 if x > 1 else x
        except:
            return np.nan

    df["CTR"] = df["CTR"].apply(parse_ctr)
    for c in ["Clicks","Impressions","Position"]:
        df[c] = pd.to_numeric(df[c], errors="coerce")

    df = df.dropna(subset=["Clicks","Impressions","CTR","Position"])
    df = df[(df["Clicks"]>=0) & (df["Impressions"]>=0) & (df["CTR"].between(0,1)) & (df["Position"]>0)]
    df = df.reset_index(drop=True)
    return df

cleaned = clean_df(df)
cleaned.info()
cleaned.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Top queries  1000 non-null   object 
 1   Clicks       1000 non-null   int64  
 2   Impressions  1000 non-null   int64  
 3   CTR          1000 non-null   float64
 4   Position     1000 non-null   float64
dtypes: float64(2), int64(2), object(1)
memory usage: 39.2+ KB


,Top queries,Clicks,Impressions,CTR,Position
0,number guessing game python,5223,14578,0.3583,1.61
1,thecleverprogrammer,2809,3456,0.8128,1.02
2,python projects with source code,2077,73380,0.0283,5.94
3,classification report in machine learning,2012,4959,0.4057,1.28
4,the clever programmer,1931,2528,0.7638,1.09


EDA

In [18]:
corr = cleaned[["Clicks","Impressions","CTR","Position"]].corr()
fig_corr = px.imshow(corr, text_auto=True, title="Correlation Matrix")
fig_corr.show()

# top 20 words in queries (quick NLP)
import re
from collections import Counter
words = []
for q in cleaned["Top queries"].astype(str):
    tokens = re.findall(r"\b[a-zA-Z]+\b", q.lower())
    words.extend([t for t in tokens if len(t) > 2])
top_words = pd.DataFrame(Counter(words).most_common(20), columns=["word","freq"])
px.bar(top_words, x="word", y="freq", title="Top 20 Words").show()

Feature Engineering

In [19]:
def engineer_features(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    out["log_clicks"] = np.log1p(out["Clicks"])
    out["log_impr"] = np.log1p(out["Impressions"])

    # log–log regression: log(CTR) ~ log(Position)
    eps = 1e-9
    x = np.log(out["Position"] + eps)
    y = np.log(out["CTR"] + eps)
    A = np.vstack([np.ones_like(x), x]).T
    coef, *_ = np.linalg.lstsq(A, y, rcond=None)
    intercept, slope = coef[0], coef[1]

    exp_ctr = np.exp(intercept + slope*np.log(out["Position"] + eps))
    out["ctr_expected"] = np.clip(exp_ctr, 0, 1)
    out["ctr_gap"] = out["CTR"] - out["ctr_expected"]

    # z-scored features for the model
    feats = out[["log_clicks","log_impr","CTR","Position","ctr_gap"]].copy()
    scaler = StandardScaler()
    out[["z_log_clicks","z_log_impr","z_CTR","z_Position","z_ctr_gap"]] = scaler.fit_transform(feats)
    return out

eng = engineer_features(cleaned)
eng.head()

,Top queries,Clicks,Impressions,CTR,Position,log_clicks,log_impr,ctr_expected,ctr_gap,z_log_clicks,z_log_impr,z_CTR,z_Position,z_ctr_gap
0,number guessing game python,5223,14578,0.3583,1.61,8.561019,9.587337,0.391063,-0.032763,5.095011,2.439091,0.802568,-0.836471,-0.403336
1,thecleverprogrammer,2809,3456,0.8128,1.02,7.940940,8.148156,0.730478,0.082322,4.264832,1.251031,3.553993,-1.044187,0.809707
2,python projects with source code,2077,73380,0.0283,5.94,7.639161,11.203420,0.065480,-0.037180,3.860803,3.773186,-1.195166,0.687951,-0.449892
3,classification report in machine learning,2012,4959,0.4057,1.28,7.607381,8.509161,0.535323,-0.129623,3.818255,1.549044,1.089515,-0.952651,-1.424268
4,the clever programmer,1931,2528,0.7638,1.09,7.566311,7.835579,0.667030,0.096770,3.763269,0.992995,3.257359,-1.019543,0.961991


Train IsolationForest & Score

In [20]:
def train_iforest(X, contamination=0.01, n_estimators=200, random_state=RANDOM_STATE):
    model = IsolationForest(
        n_estimators=n_estimators,
        contamination=contamination,
        random_state=random_state,
        n_jobs=-1
    )
    model.fit(X)
    return model

feature_cols = ["z_log_clicks","z_log_impr","z_CTR","z_Position","z_ctr_gap"]
X = eng[feature_cols]
model = train_iforest(X, contamination=0.01, n_estimators=200)

# decision_function: higher = more normal, lower = more anomalous
eng["anomaly_score"] = model.decision_function(X)
eng["prediction"] = model.predict(X)
anomaly_rate = (eng["prediction"] == -1).mean()
print(f"Model anomaly rate: {anomaly_rate:.2%}")

Model anomaly rate: 1.00%


Score Distribution & Top Anomalies

In [21]:
px.histogram(eng, x="anomaly_score", nbins=50,
             title="IsolationForest Decision Score (lower = more anomalous)").show()

display_cols = ["Top queries","Clicks","Impressions","CTR","Position","ctr_expected","ctr_gap","anomaly_score"]
top_anoms = (eng[eng["prediction"] == -1]
             .sort_values("anomaly_score")
             .head(100)[display_cols]
            ).copy()

# pretty percentages for view (we'll keep raw for CSV)
pretty = top_anoms.copy()
pretty["CTR"] = (pretty["CTR"]*100).round(2).astype(str) + "%"
pretty["ctr_expected"] = (pretty["ctr_expected"]*100).round(2).astype(str) + "%"
pretty["ctr_gap"] = (pretty["ctr_gap"]*100).round(2).astype(str) + "%"
pretty.head(20)

,Top queries,Clicks,Impressions,CTR,Position,ctr_expected,ctr_gap,anomaly_score
1,thecleverprogrammer,2809,3456,81.28%,1.02,73.05%,8.23%,-0.066117
91,facebook programming languages,346,36055,0.96%,1.58,40.13%,-39.17%,-0.060032
4,the clever programmer,1931,2528,76.38%,1.09,66.7%,9.68%,-0.047480
2,python projects with source code,2077,73380,2.83%,5.94,6.55%,-3.72%,-0.042803
167,text to handwriting,222,11283,1.97%,28.52,0.76%,1.21%,-0.035911
0,number guessing game python,5223,14578,35.83%,1.61,39.11%,-3.28%,-0.034446
929,python turtle,52,18228,0.29%,18.75,1.36%,-1.07%,-0.025927
858,water quality analysis,56,7359,0.76%,27.56,0.8%,-0.04%,-0.020697
5,standard scaler in machine learning,1559,7292,21.38%,1.53,41.93%,-20.55%,-0.020439
3,classification report in machine learning,2012,4959,40.57%,1.28,53.53%,-12.96%,-0.009622


Evaluation

In [22]:
def proxy_precision_at_k(df_scored: pd.DataFrame, scores_col="anomaly_score", k=50):
    p75_impr = np.percentile(df_scored["Impressions"], 75)
    p25_ctr = np.percentile(df_scored["CTR"], 25)
    proxy_label = (df_scored["Impressions"] >= p75_impr) & (df_scored["CTR"] <= p25_ctr)

    order = np.argsort(df_scored[scores_col].values)  # ascending = more anomalous
    topk = order[:min(k, len(order))]
    prec = proxy_label.iloc[topk].mean() if len(topk) else np.nan
    return float(prec), int(min(k, len(order)))

prec_k, used_k = proxy_precision_at_k(eng, k=100)
print(f"Proxy Precision@{used_k}: {prec_k:.2%}  (sanity check only)")

Proxy Precision@100: 30.00%  (sanity check only)


In [23]:
anomalies_csv = (eng[eng["prediction"] == -1]
                 .sort_values("anomaly_score")
                 [display_cols])
anomalies_csv.to_csv("anomalies.csv", index=False)
print("Saved anomalies.csv")

from google.colab import files
files.download("anomalies.csv")

Saved anomalies.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Executive Notes

In [24]:
summary = f"""
Rows: {len(cleaned)}
Model: IsolationForest (n_estimators=200, contamination=1%)
Anomaly rate: {(eng["prediction"]==-1).mean():.2%}
Proxy Precision@100: {prec_k:.2%} (heuristic: high impressions & low CTR)

Signals:
- Expected CTR from Position via log–log regression; CTR gap highlights under-performers at a given rank.
- Features: log_clicks, log_impr, CTR, Position, ctr_gap (z-scored).

Actions:
- Review top anomalies (high impressions + negative CTR gap), refresh snippets/titles, check SERP competition,
  and consider A/B tests. Track CTR change post-fix.
"""
print(summary)


Rows: 1000
Model: IsolationForest (n_estimators=200, contamination=1%)
Anomaly rate: 1.00%
Proxy Precision@100: 30.00% (heuristic: high impressions & low CTR)

Signals:
- Expected CTR from Position via log–log regression; CTR gap highlights under-performers at a given rank.
- Features: log_clicks, log_impr, CTR, Position, ctr_gap (z-scored).

Actions:
- Review top anomalies (high impressions + negative CTR gap), refresh snippets/titles, check SERP competition,
  and consider A/B tests. Track CTR change post-fix.

